In [1]:
# Librerias
import warnings
warnings.filterwarnings("ignore")
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance

In [2]:
df_original = pd.read_csv("../data/raw/power_tetouan_city_original.csv")
df_modified = pd.read_csv("../data/raw/power_tetouan_city_modified.csv")

In [3]:
df_original.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


df_modified.head()

In [4]:
df_modified.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption,mixed_type_col
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,NaN,34055.6962,16128.87538,20240.96386,624
1,1/1/2017 0:10,6.414,74.5,0.083,0.07,0.085,29814.68354,19375.07599,20131.08434,811
2,1/1/2017 0:20,6.313,74.5,0.08,0.062,0.1,29128.10127,19006.68693,19668.43373,unknown
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711,bad
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.6962,17872.34043,18442.40964,879


In [5]:
# Renombrar variables 
df_modified = df_modified.rename(columns={'Zone 1 Power Consumption': 'PC_Zone1', 
                        'Zone 2  Power Consumption': 'PC_Zone2',
                        'Zone 3  Power Consumption': 'PC_Zone3'})

In [6]:
# Información del dataset
df_modified.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53464 entries, 0 to 53463
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DateTime               52977 non-null  object
 1   Temperature            52880 non-null  object
 2   Humidity               52826 non-null  object
 3   Wind Speed             52846 non-null  object
 4   general diffuse flows  52838 non-null  object
 5   diffuse flows          52849 non-null  object
 6   PC_Zone1               52861 non-null  object
 7   PC_Zone2               52839 non-null  object
 8   PC_Zone3               52889 non-null  object
 9   mixed_type_col         48162 non-null  object
dtypes: object(10)
memory usage: 4.1+ MB


In [7]:
# Drop NA
df_clean = df_modified.dropna()

In [8]:
df_clean = df_clean.drop_duplicates()

In [9]:
# Información del dataset limpio
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43505 entries, 1 to 53463
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DateTime               43505 non-null  object
 1   Temperature            43505 non-null  object
 2   Humidity               43505 non-null  object
 3   Wind Speed             43505 non-null  object
 4   general diffuse flows  43505 non-null  object
 5   diffuse flows          43505 non-null  object
 6   PC_Zone1               43505 non-null  object
 7   PC_Zone2               43505 non-null  object
 8   PC_Zone3               43505 non-null  object
 9   mixed_type_col         43505 non-null  object
dtypes: object(10)
memory usage: 3.7+ MB


In [10]:
# Descrbir el dataset
df_clean.describe()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,PC_Zone1,PC_Zone2,PC_Zone3,mixed_type_col
count,43505,43505,43505,43505,43505,43505,43505,43505,43505,43505
unique,42875,5054,5873,1049,10834,10804,26483,27939,22549,1002
top,NAN,?,84.6,0.082,0.062,0.115,?,error,error,unknown
freq,18,50,159,1784,1221,976,49,44,45,4939


In [11]:
df_clean.loc[:,df_clean.columns != "DateTime"] = df_clean.loc[:,df_clean.columns != "DateTime"].apply(pd.to_numeric, errors="coerce")

In [12]:
df_clean.dtypes

DateTime                 object
Temperature              object
Humidity                 object
Wind Speed               object
general diffuse flows    object
diffuse flows            object
PC_Zone1                 object
PC_Zone2                 object
PC_Zone3                 object
mixed_type_col           object
dtype: object